## Librerías

In [10]:
import os
import librosa
import librosa.display
import pandas as pd

import tensorflow as tf
import keras
import matplotlib as plt
import sklearn


In [1]:
import numpy as np
import pickle

## Lectura de datos

In [ ]:
audio_fpath = "../desafio/data/Train/"
audio_clips = os.listdir(audio_fpath)
print("No. of .wav files in audio folder = ",len(audio_clips))

In [ ]:
audio_clips[:5]

In [ ]:
x , sr = librosa.load(f"../desafio/data/Train/{audio_clips[0]}")

In [ ]:
x.shape,sr

In [ ]:
librosa.display.waveshow(x, sr=sr)

In [ ]:
X = librosa.stft(x)
Xdb = librosa.amplitude_to_db(abs(X))

In [ ]:
librosa.display.specshow(Xdb,sr=sr)

In [ ]:
Xdb

## Procesamiento

In [ ]:
#X_test = []
#for clip in audio_clips:
#    x , sr = librosa.load(f"../desafio/data/Train/{clip}")
#    X = librosa.stft(x)
#    Xdb = librosa.amplitude_to_db(abs(X))
#    X_test.append(Xdb)

In [ ]:
#file = open("data.obj", "wb")
#data = pickle.dump(X_test, file)
#file.close()

In [ ]:
file = open("data.obj", "rb")
data = pickle.load(file)
file.close()

In [ ]:
maxshape = 0
for s in data:
    if s.shape[1] >= maxshape:
        maxshape = s.shape[1]
data[0].shape[0],maxshape

In [ ]:
datareshape = []
for item in data:
    i,j = item.shape
    fill_left = np.floor((maxshape-j)/2)
    fill_right = np.ceil((maxshape-j)/2)
    datareshape.append(np.pad(item, ((0,0),(int(fill_left),int(fill_right))), mode='constant', constant_values=np.min(item)))

In [ ]:
librosa.display.specshow(datareshape[0],sr=sr)

In [ ]:
npdata = np.asarray(datareshape)

In [ ]:
del data,datareshape

In [ ]:
#file = open("npdata.obj", "wb")
#data = pickle.dump(npdata, file)
#file.close()

In [ ]:
file = open("npdata.obj", "rb")
npdata = pickle.load(file)
file.close()

In [ ]:
npdata.shape

In [ ]:
#data_tf = tf.convert_to_tensor(npdata, np.float32)

## Data augmentation

In [ ]:
import nlpaug.flow as naf
import nlpaug.augmenter.spectrogram as nas
from nlpaug.util.audio.visualizer import AudioVisualizer


test = npdata[1]
flow = naf.Sequential([
    nas.FrequencyMaskingAug(zone=(0,1)), 
    nas.LoudnessAug(coverage=1,factor=(0.8,1.5))
])
aug_test = flow.augment(test)


librosa.display.specshow(test,sr=sr)

In [ ]:
reshape_test = np.reshape(np.asarray(aug_test), (1025, 537))
librosa.display.specshow(reshape_test,sr=sr)

In [ ]:
def augment(image):
    flow = naf.Sequential([
        nas.FrequencyMaskingAug(zone=(0,1)), 
        nas.FrequencyMaskingAug(zone=(0,1)), 
        nas.TimeMaskingAug(coverage=.00001), 
        nas.LoudnessAug(coverage=.0001)
    ])
    temp_image = flow.augment(image)
    aug_image = np.reshape(np.asarray(temp_image), image.shape)
    return aug_image


In [ ]:
#from IPython.display import clear_output
#import time

In [ ]:
#for i in range(500):
#    librosa.display.specshow(augment_data[i],sr=sr)
#    plt.pyplot.show()
#    clear_output(wait = True)
#    time.sleep(1)

In [ ]:
extended_data = npdata
for i in range(5):
    augment_data = np.array(list(map(augment, npdata)))
    extended_data = np.append(extended_data, augment_data, axis=0)
del augment_data,npdata

In [ ]:
extended_data.shape

In [ ]:
#file = open("extended_data.obj", "wb")
#pickle.dump(extended_data, file)
#file.close()

In [2]:
file = open("extended_data.obj", "rb")
extended_data = pickle.load(file)
file.close()

In [ ]:
s

In [ ]:
s.shape

In [ ]:
np.mean(s, axis = 0).shape

In [ ]:
extended_data.shape

In [ ]:
mean = np.mean(extended_data,axis=0)


In [ ]:
std = np.std(extended_data,axis=0)

In [ ]:
#file = open("std.obj", "wb")
#pickle.dump(std, file)
#file.close()

In [4]:
file = open("mean.obj", "rb")
mean = pickle.load(file)
file.close()

In [7]:
file = open("std.obj", "rb")
std = pickle.load(file)
file.close()

In [ ]:
# file = open("mean.obj", "wb")
# pickle.dump(mean, file)
# file.close()

In [8]:
extended_data=extended_data/std

In [9]:
#file = open("normalized.obj", "wb")
#pickle.dump(extended_data, file)
#file.close()

In [ ]:
file = open("normalized.obj", "rb")
normalized = pickle.load(file)
file.close()

## Lectura de vector objetivo

In [11]:
df_genre = pd.read_csv('../desafio/data/Train.csv')

In [12]:
df_genre.head()

,Id,Expected
0,01216683570.wav,0 2
1,00433588573.wav,0 2
2,00381534896.wav,0 2
3,01635825413.wav,0 2
4,00325117692.wav,0 2


In [13]:
df_genre['Expected'] = df_genre['Expected'].apply(lambda x: x[0])

In [14]:
df_genre.head()

,Id,Expected
0,01216683570.wav,0
1,00433588573.wav,0
2,00381534896.wav,0
3,01635825413.wav,0
4,00325117692.wav,0


In [15]:
y1 = np.array(list(map(float, df_genre["Expected"])))
print(y1)

[0. 0. 0. ... 1. 1. 1.]


In [16]:
extended_y = y1
for i in range(5):
    extended_y=np.append(extended_y, y1, axis=0)
extended_y.shape

(8370,)

In [17]:
#from sklearn.model_selection import train_test_split
#
#X1_tr, X1_val, y1_tr, y1_val = train_test_split(
#    extended_data, extended_y, test_size=0.2)

In [18]:
#del extended_data,extended_y,y1

## Red Masculino/Femenino

In [37]:
model = keras.models.Sequential([
    keras.layers.Conv2D(32, 3, strides=2, activation='relu', input_shape = [1025,537,1]),
    keras.layers.MaxPooling2D(pool_size=(3, 3)),
    keras.layers.Conv2D(64, 2, strides=2, activation='relu'),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),
    keras.layers.Conv2D(128, 2, strides=2, activation='relu'),
    keras.layers.Flatten(),
    keras.layers.Dense(50, activation='relu'),
    keras.layers.Dense(1, activation ='sigmoid')
])

In [38]:
model.compile(optimizer=keras.optimizers.Adam(0.001),
             loss = 'binary_crossentropy',
             metrics=['accuracy'])

In [39]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_15 (Conv2D)          (None, 512, 268, 32)      320       
                                                                 
 max_pooling2d_11 (MaxPoolin  (None, 170, 89, 32)      0         
 g2D)                                                            
                                                                 
 conv2d_16 (Conv2D)          (None, 85, 44, 64)        8256      
                                                                 
 max_pooling2d_12 (MaxPoolin  (None, 42, 22, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_17 (Conv2D)          (None, 21, 11, 128)       32896     
                                                                 
 flatten_4 (Flatten)         (None, 29568)            

In [31]:
#from sklearn.preprocessing import StandardScaler
#
#scaler_x1 = StandardScaler()
#scaler_x1.fit(X1)
#x1_tr = scaler_x1.transform(X1_tr)
#x1_val = scaler_x1.transform(X1_val)

In [40]:
epochs = 50
callback = keras.callbacks.EarlyStopping(monitor='val_loss',
                                            patience=5,
                                            restore_best_weights=True)

In [41]:
history = model.fit(extended_data,extended_y, epochs=epochs, validation_split=0.1,callbacks=callback)

Epoch 1/50
236/236 [==============================] - 174s 735ms/step - loss: 0.6928 - accuracy: 0.5364 - val_loss: 0.7110 - val_accuracy: 0.3214
Epoch 2/50
236/236 [==============================] - 165s 700ms/step - loss: 0.6806 - accuracy: 0.5615 - val_loss: 0.7034 - val_accuracy: 0.5161
Epoch 3/50
236/236 [==============================] - 165s 698ms/step - loss: 0.6374 - accuracy: 0.6320 - val_loss: 0.7257 - val_accuracy: 0.5412
Epoch 4/50
236/236 [==============================] - 161s 683ms/step - loss: 0.5149 - accuracy: 0.7387 - val_loss: 0.5410 - val_accuracy: 0.7121
Epoch 5/50
236/236 [==============================] - 162s 688ms/step - loss: 0.3273 - accuracy: 0.8505 - val_loss: 0.3672 - val_accuracy: 0.8351
Epoch 6/50
236/236 [==============================] - 163s 692ms/step - loss: 0.1671 - accuracy: 0.9323 - val_loss: 0.3556 - val_accuracy: 0.8542
Epoch 7/50
236/236 [==============================] - 163s 689ms/step - loss: 0.0980 - accuracy: 0.9643 - val_loss: 0.3590 -

In [42]:
model.save("../Redes/desafio/models/Gender2w/normalization")

INFO:tensorflow:Assets written to: ../Redes/desafio/models/Gender2w/normalization\assets


INFO:tensorflow:Assets written to: ../Redes/desafio/models/Gender2w/normalization\assets


In [43]:
model.save_weights("../Redes/desafio/models/Gender2_weights")

In [ ]:
keras.models.load_model("../Redes/desafio/models/Gender2w/normalization")